In [1]:
import shutil
import os
import os.path as op
from collections import OrderedDict

from eptools.people import (get_profiles_registry,
                            fetch_profiles_files,
                            contact_from_dict)

from eptools.badges.factory import (BadgeFactory, 
                                    get_badge_role,
                                    prepare_badge_pdf,
                                    get_badge_template_file)

In [2]:
#INKSCAPE_BINPATH = '/Applications/Inkscape.app/Contents/Resources/bin/inkscape-bin'

output_dir = op.expanduser('/vagrant/eps_badges2')
os.makedirs(output_dir, exist_ok=True)

CURRENT_EUROPYTHON_FIRST_TICKET_ID = 3732

factory = BadgeFactory(output_dir)

In [3]:
fetch_data = False

# fetch the data from the DB
if fetch_data:
    profiles_file, talks_file = fetch_profiles_files()
    print(profiles_file, talks_file)

In [4]:
def create_badge(contact, roles, factory):

    # create the SVG file
    svg_file = factory.generate_badge_svg(contact, roles)

    # convert the SVG to PDF
    pdf_file = prepare_badge_pdf(svg_file, doublepages=False)

    # make both badge faces and save the result in its place
    outdir = op.join(factory.out_dir, get_badge_role(roles).name)
    os.makedirs(outdir, exist_ok=True)
    
    return outdir, pdf_file


import hashlib

def file_cmp(file1, file2, hash_func=hashlib.sha224):
    os.utime(file1)
    os.utime(file2) # touch
    
    hash1 = hash_func(open(file1, 'rb').read()).hexdigest()
    hash2 = hash_func(open(file2, 'rb').read()).hexdigest()
    return hash1 == hash2

In [5]:
# load and prepared the profiles
profiles = get_profiles_registry()

In [ ]:
new_badges = []
for profile in profiles:
   
    if profile.pop('frozen'):
        continue

    if profile['id'] < CURRENT_EUROPYTHON_FIRST_TICKET_ID:
        continue
    
    contact = contact_from_dict(profile)

    roles   = list(profiles.get_roles_of(contact.email, 
                                         name=contact.name, 
                                         surname=contact.surname))
       
    outdir, pdf_file = create_badge(contact, roles, factory)
    
    result_file = op.join(outdir, op.basename(pdf_file))

    move = False
    #if op.exists(result_file):
    #    if not file_cmp(pdf_file, result_file): # not equal
    #        move = True
    #else:
    #    move = True
    if not op.exists(result_file):
        move = True

    if move:
        _ = shutil.move(pdf_file, result_file)
        new_badges.append(result_file)
        print(' ✓')
    else:
        print(' X')

 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓
 ✓


In [ ]:
import subprocess
from glob import glob
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [ ]:
# print 2x2 sheets doublepages with profile badges

import numpy as np

badge_files = glob(op.join(output_dir, 'out', '**', '*.pdf'))

cmd1 = 'pdfjam {input} --nup 2x2 --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'
cmd2 = 'pdfjam {input1} {input2} --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print', 'profiles', 'ep2017_badges_{}.pdf')

blank_badge = '/vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf'

for idx, badges in enumerate(grouper(badge_files, 4, fillvalue=blank_badge)):
    
    badges_front = np.array(badges)
    badges_rear = badges_front[[1, 0, 3, 2]]
    
    output_front = output_template.format(str(idx) + '_front')
    output_rear = output_template.format(str(idx) + '_rear')
    output = output_template.format(str(idx))
    
    make_badge_front = cmd1.format(input=' '.join(badges_front), output=output_front)
    make_badge_rear  = cmd1.format(input=' '.join(badges_rear), output=output_rear)
    doublepages      = cmd2.format(input1=output_front, input2=output_rear, output=output)
    
    print(make_badge_front)
    subprocess.check_call(make_badge_front, shell=True)

    print(make_badge_rear)
    subprocess.check_call(make_badge_rear, shell=True)
    
    print(doublepages)
    subprocess.check_call(doublepages, shell=True)
    
    os.remove(output_front)
    os.remove(output_rear)
    
    

In [45]:
# print one 2x2 sheet doublepages with blank daily badges

import numpy as np

badge_files = glob('/vagrant/ep-tools/eptools/badges/data/ep2017/blanks/*.pdf')

cmd1 = 'pdfjam {input} --nup 2x2 --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'
cmd2 = 'pdfjam {input1} {input2} --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print', 'blanks', 'ep2017_badges_{}.pdf')

for idx, badges in enumerate(grouper(badge_files, 4, fillvalue='')):
    
    badges_front = np.array(badges)
    badges_rear = badges_front[[1, 0, 3, 2]]
    
    output_front = output_template.format(str(idx) + '_front')
    output_rear = output_template.format(str(idx) + '_rear')
    output = output_template.format(str(idx))
    
    make_badge_front = cmd1.format(input=' '.join(badges_front), output=output_front)
    make_badge_rear  = cmd1.format(input=' '.join(badges_rear), output=output_rear)
    doublepages      = cmd2.format(input1=output_front, input2=output_rear, output=output)
    
    print(make_badge_front)
    subprocess.check_call(make_badge_front, shell=True)

    print(make_badge_rear)
    subprocess.check_call(make_badge_rear, shell=True)
    
    print(doublepages)
    subprocess.check_call(doublepages, shell=True)
    
    os.remove(output_front)
    os.remove(output_rear)
    
    

pdfjam /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/friday.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/monday.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/saturday.pdf --nup 2x2 --papersize "{23cm,33cm}" --noautoscale true -o /vagrant/eps_badges2/to_print/blanks/ep2017_badges_0_front.pdf
pdfjam /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/monday.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/friday.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/saturday.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf --nup 2x2 --papersize "{23cm,33cm}" --noautoscale true -o /vagrant/eps_badges2/to_print/blanks/ep2017_badges_0_rear.pdf
pdfjam /vagrant/eps_badges2/to_print/blanks/ep2017_badges_0_front.pdf /vagrant/eps_badges2/to_print/blanks/ep2017_badges_0_rear.pdf --papersize "{23cm,33cm}" --noautoscale true -o /vagrant/eps_badges2/to_print/blanks/ep2017_bad

In [46]:
# print one 2x2 sheet doublepages with 4 blank participant badges

import numpy as np

badge_files = ['/vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf'] * 4

cmd1 = 'pdfjam {input} --nup 2x2 --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'
cmd2 = 'pdfjam {input1} {input2} --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print', 'blanks', 'ep2017_badges_{}.pdf')

for idx, badges in enumerate(grouper(badge_files, 4, fillvalue='')):
    
    badges_front = np.array(badges)
    badges_rear = badges_front[[1, 0, 3, 2]]
    
    output_front = output_template.format(str(idx) + '_front')
    output_rear = output_template.format(str(idx) + '_rear')
    output = output_template.format(str(idx))
    
    make_badge_front = cmd1.format(input=' '.join(badges_front), output=output_front)
    make_badge_rear  = cmd1.format(input=' '.join(badges_rear), output=output_rear)
    doublepages      = cmd2.format(input1=output_front, input2=output_rear, output=output)
    
    print(make_badge_front)
    subprocess.check_call(make_badge_front, shell=True)

    print(make_badge_rear)
    subprocess.check_call(make_badge_rear, shell=True)
    
    print(doublepages)
    subprocess.check_call(doublepages, shell=True)
    
    os.remove(output_front)
    os.remove(output_rear)
    
    

pdfjam /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf --nup 2x2 --papersize "{23cm,33cm}" --noautoscale true -o /vagrant/eps_badges2/to_print/blanks/ep2017_badges_0_front.pdf
pdfjam /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf /vagrant/ep-tools/eptools/badges/data/ep2017/blanks/participant.pdf --nup 2x2 --papersize "{23cm,33cm}" --noautoscale true -o /vagrant/eps_badges2/to_print/blanks/ep2017_badges_0_rear.pdf
pdfjam /vagrant/eps_badges2/to_print/blanks/ep2017_badges_0_front.pdf /vagrant/eps_badges2/to_print/blanks/ep2017_badges_0_rear.pdf --papersize "{23cm,33cm}" --noautoscale true -o /vagrant/eps_badges2/

# Snippets

In [14]:
badge_files = glob(op.join(output_dir, 'out', '**', '*.pdf'))

cmd = 'pdfjam {input} --nup 1x1 --doublepages true --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print_1x1_big', 'profiles', '{}')

for idx, badge in enumerate(badge_files): 
    
    output = output_template.format(op.basename(badge))
    
    make_badge = cmd.format(input=badge, output=output)
    
    print(make_badge)
    subprocess.call(make_badge, shell=True)

pdfjam /home/alexandre/Desktop/eps_badges/out/organizer/badge_ep2017_organizer_3968.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_organizer_3968.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/organizer/badge_ep2017_organizer_4702.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_organizer_4702.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/organizer/badge_ep2017_organizer_3792.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_organizer_3792.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/organizer/badge_ep2017_organizer_4382.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/

pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4007.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4007.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4960.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4960.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4753.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4753.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4747.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep201

pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4028.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4028.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_5236.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_5236.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4330.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4330.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4154.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep201

pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4518.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4518.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4276.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4276.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4205.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4205.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_5051.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep201

pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4845.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4845.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_3812.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_3812.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4501.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_attendee_4501.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/attendee/badge_ep2017_attendee_4175.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep201

...

pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4586.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4586.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4621.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4621.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_5055.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_5055.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_3836.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_3

pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_3769.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_3769.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4528.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4528.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4365.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4365.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4016.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4

pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_3942.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_3942.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_3975.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_3975.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4021.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4021.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_3979.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_3

pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4857.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4857.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4058.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4058.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4542.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4542.pdf
pdfjam /home/alexandre/Desktop/eps_badges/out/speaker/badge_ep2017_speaker_4548.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_speaker_4

pdfjam /home/alexandre/Desktop/eps_badges/out/trainer/badge_ep2017_trainer_4045.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/profiles/badge_ep2017_trainer_4045.pdf


In [20]:
badge_files = glob(op.join(output_dir, 'out', '**', '*.pdf'))

cmd1 = 'pdfjam {input} --nup 1x2 --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'
cmd2 = 'pdfjam {input} --doublepages true --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print', 'profiles', 'ep2017_badges_{}.pdf')

for idx, badges in enumerate(grouper(badge_files, 2, fillvalue='')):
    
    output = output_template.format(idx)
    
    make_badge = cmd1.format(input=' '.join(badges), output=output)
    doublepages = cmd2.format(input=output, output=output)
    
    print(make_badge)
    subprocess.call(make_badge, shell=True)
    
    print(doublepages)
    subprocess.call(doublepages, shell=True)
    break
    
    

pdfjam /vagrant/eps_badges2/out/attendee/badge_ep2017_attendee_5162.pdf /vagrant/eps_badges2/out/attendee/badge_ep2017_attendee_5163.pdf --nup 1x2 --papersize "{23cm,33cm}" --noautoscale true -o /vagrant/eps_badges2/to_print/profiles/ep2017_badges_0.pdf
pdfjam /vagrant/eps_badges2/to_print/profiles/ep2017_badges_0.pdf --doublepages true --noautoscale true -o /vagrant/eps_badges2/to_print/profiles/ep2017_badges_0.pdf


In [15]:
badge_files = glob('/home/alexandre/Projects/ep-tools/eptools/badges/data/ep2017/blanks/*.pdf')

cmd = 'pdfjam {input} --nup 1x1 --doublepages true --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print_1x1_big', 'blanks', '{}')

for idx, badge in enumerate(badge_files): 
    
    output = output_template.format(op.basename(badge))
    
    make_badge = cmd.format(input=badge, output=output)
    
    print(make_badge)
    subprocess.call(make_badge, shell=True)

pdfjam /home/alexandre/Projects/ep-tools/eptools/badges/data/ep2017/blanks/wednesday.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/blanks/wednesday.pdf
pdfjam /home/alexandre/Projects/ep-tools/eptools/badges/data/ep2017/blanks/friday.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/blanks/friday.pdf
pdfjam /home/alexandre/Projects/ep-tools/eptools/badges/data/ep2017/blanks/thursday.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/blanks/thursday.pdf
pdfjam /home/alexandre/Projects/ep-tools/eptools/badges/data/ep2017/blanks/monday.pdf --nup 1x1 --doublepages true --papersize "{23cm,33cm}" --noautoscale true -o /home/alexandre/Desktop/eps_badges/to_print_1x1_big/blanks/monday.pdf
pdfjam /home/alexandre/Projects/ep-tools/eptools/badges/da